In [ ]:
!pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense
import keras_tuner as kf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv('/content/Churn_Modelling.csv')

In [ ]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
df.drop(columns = ['RowNumber'	,'CustomerId',	'Surname'], inplace=True)

In [ ]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
df = pd.get_dummies(data = df,columns=['Geography','Gender'])
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,True,False,False,True,False
1,608,41,1,83807.86,1,0,1,112542.58,0,False,False,True,True,False
2,502,42,8,159660.80,3,1,0,113931.57,1,True,False,False,True,False
3,699,39,1,0.00,2,0,0,93826.63,0,True,False,False,True,False
4,850,43,2,125510.82,1,1,1,79084.10,0,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,True,False,False,False,True
9996,516,35,10,57369.61,1,1,1,101699.77,0,True,False,False,False,True
9997,709,36,7,0.00,1,0,1,42085.58,1,True,False,False,True,False
9998,772,42,3,75075.31,2,1,0,92888.52,1,False,True,False,False,True


In [ ]:
x = df.drop(columns = ['Exited'])
y = df['Exited']

In [ ]:
sc = StandardScaler()

x = sc.fit_transform(x)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.20, random_state=1)

In [ ]:
# Before hyperparameter tuning: initially use random parameters only for comparision
ann = Sequential()# random weights are initialized so every time you will get different accuracy and loss

ann.add(Dense(units = 10, activation = 'relu'))# hidden layer
ann.add(Dense(units = 1, activation = 'sigmoid'))# output layer

ann.compile(optimizer = 'adadelta', loss = 'binary_crossentropy', metrics ='accuracy')

ann.fit(xtrain,ytrain,epochs = 100)

Epoch 1/100
250/250 [==============================] - 3s 6ms/step - loss: 1.2152 - accuracy: 0.2786
Epoch 2/100
250/250 [==============================] - 2s 7ms/step - loss: 1.2128 - accuracy: 0.2791
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 1.2102 - accuracy: 0.2799
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 1.2076 - accuracy: 0.2800
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 1.2048 - accuracy: 0.2804
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 1.2020 - accuracy: 0.2809
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 1.1991 - accuracy: 0.2820
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 1.1961 - accuracy: 0.2829
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 1.1932 - accuracy: 0.2830
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 1.1902 - accura

In [ ]:
ypred = ann.predict(xtest)
ypred = ypred > 0.5
ypred

63/63 [==============================] - 0s 1ms/step


array([[ True],
       [False],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

In [ ]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.78      0.31      0.44      1585
           1       0.20      0.66      0.31       415

    accuracy                           0.38      2000
   macro avg       0.49      0.49      0.38      2000
weighted avg       0.66      0.38      0.41      2000



In [ ]:
ann.evaluate(xtrain,ytrain)

250/250 [==============================] - 0s 1ms/step - loss: 0.9417 - accuracy: 0.3846


[0.941658079624176, 0.38462498784065247]

# Hyperparameter tuning


In [ ]:
list(range(8,128,8))

[8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120]

In [ ]:
# This function optimizes No. of layers, No. of neurons in each layer, activation function and optimizer
def hyper(hp): #This defines a function named hyper that takes a single argument hp.
    model = Sequential()
    counter = 0 #Initializes a counter variable to keep track of the current layer being added to the model.
    for i in range(hp.Int('num_layers', min_value = 1,max_value=10)):  # Find No. of hiddern layers: Loops through a range of layers,
        # where the number of layers is a hyperparameter named 'num_layers' with a minimum value of 1 and a maximum value of 10.
        # No. of Hidden layers are 1 to 10. In each layer need to add no. of neurons and activation function. So following if-else code
        if counter == 0: # If block to find no. of hidden layers, will be executed only once
            model.add(Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                            activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid']),
                            input_dim = 13))
            #Adds the first layer with input dimensions specified by input_dim=13.
            #The number of units ('units') and activation function ('activation') are hyperparameters,
            #and their values are determined by the hp object.
        else: # else block to find no of neurons and activation function
          model.add(Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
              activation = hp.Choice('activation'+str(i),values = ['relu','tanh','sigmoid'])))

              #Adds subsequent layers (after the first layer) with hyperparameter-specified units and activation functions.

    counter+=1   #Increments the counter to keep track of the layers added.
    model.add(Dense(units = 1,activation='sigmoid'))   #Adds the output layer with a single unit and sigmoid activation function, typical for binary classification problems.
    model.compile(optimizer = hp.Choice('optimizer',values = ['adam','rmsprop','sgd']),
                loss = 'binary_crossentropy',metrics = 'accuracy')
    return model  #Returns the compiled model.

In [ ]:
tuner = kf.RandomSearch(hyper,
                        objective = 'val_accuracy',
                        max_trials = 3)

In [ ]:
tuner.search(xtrain,ytrain,epochs = 5, validation_data = (xtest,ytest))
# search() is similar to fit()

In [ ]:
tuner.get_best_hyperparameters()[0].values # will give first model where accuracy is high

{'num_layers': 3,
 'units0': 72,
 'activation0': 'relu',
 'optimizer': 'rmsprop',
 'units1': 96,
 'activation1': 'relu',
 'units2': 104,
 'activation2': 'relu',
 'units3': 56,
 'activation3': 'relu',
 'units4': 32,
 'activation4': 'relu',
 'units5': 56,
 'activation5': 'sigmoid',
 'units6': 72,
 'activation6': 'sigmoid',
 'units7': 48,
 'activation7': 'relu',
 'units8': 48,
 'activation8': 'tanh',
 'units9': 24,
 'activation9': 'tanh'}

In [ ]:
tuned_model = tuner.get_best_models(num_models=1)[0]
# tuned_model.build(x.shape)

In [ ]:
tuned_model.fit(xtrain,ytrain,epochs=50, validation_data=(xtest,ytest))

Epoch 1/50
250/250 [==============================] - 2s 4ms/step - loss: 0.3337 - accuracy: 0.8600 - val_loss: 0.3365 - val_accuracy: 0.8645
Epoch 2/50
250/250 [==============================] - 1s 4ms/step - loss: 0.3301 - accuracy: 0.8631 - val_loss: 0.3405 - val_accuracy: 0.8600
Epoch 3/50
250/250 [==============================] - 1s 4ms/step - loss: 0.3275 - accuracy: 0.8666 - val_loss: 0.3452 - val_accuracy: 0.8575
Epoch 4/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3237 - accuracy: 0.8624 - val_loss: 0.3384 - val_accuracy: 0.8655
Epoch 5/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3205 - accuracy: 0.8677 - val_loss: 0.3437 - val_accuracy: 0.8570
Epoch 6/50
250/250 [==============================] - 1s 2ms/step - loss: 0.3182 - accuracy: 0.8691 - val_loss: 0.3432 - val_accuracy: 0.8615
Epoch 7/50
250/250 [==============================] - 1s 3ms/step - loss: 0.3152 - accuracy: 0.8700 - val_loss: 0.3453 - val_accuracy: 0.8605
Epoch 

In [ ]:
tuned_model.evaluate(xtrain,ytrain) # Training accuracy

250/250 [==============================] - 0s 2ms/step - loss: 0.1837 - accuracy: 0.9224


[0.18367871642112732, 0.9223750233650208]

In [ ]:
tuned_model.evaluate(xtest,ytest)# Testing accuracy

63/63 [==============================] - 0s 2ms/step - loss: 0.5423 - accuracy: 0.8355


[0.5423070788383484, 0.8355000019073486]